In [ ]:
"""
20.3 Модели базы данных
Все ORM-таблицы будут наследоваться от объекта Table.
Данный объект является представлением таблицы базы данных.
Он создает уникальный экземпляр самого себя на основе имени объекта и необязательного имени схемы.
Схемой, в данном случае, является одна из множества форм: это может быть имя схемы конкретной базы данных (например, схемы PostgreSQL), именованные родственные базы данных (например, доступ к другим базам данных на том же сервере), а также ряд других концепций. То есть, указывая определенную схему для таблицы, обращение к таблице будет schema.table. Все описанные нами таблицы хранятся в объекте Metadata. Данные объект, фактически, является простой коллекцией таблиц и связанных с ними схем.
Создадим метадату и опишем ORM модель.
Два основных аргумента модели - это имя таблицы и метадата, с которой она будет связана.
Остальные позиционные аргументы - это в основном объекты-колонки, описывающие каждый столбец:
"""
    
from sqlalchemy import MetaData, Table, Column, Integer, String, \
   create_engine
from sqlalchemy.orm import mapper, sessionmaker

engine = create_engine('sqlite:///sqlite_python.db')
Session = sessionmaker(bind=engine)
session = Session()
metadata = MetaData()
users = Table('user', metadata,
             Column('id', Integer, primary_key=True),
             Column('name', String(16), nullable=False),
             Column('email', String(60)),
             Column('login', String(50), nullable=False)
             )


class User(object):
   def __init__(self, name, email, login):
       self.name = name
       self.email = email
       self.login = login

   def __repr__(self):
       return f"{self.name}, {self.email}, {self.login}"


mapper(User, users)
metadata.create_all(bind=engine)
 
"""Описанная таблица имеет имя users и содержит 4 столбца, каждая таблица должна иметь первичный ключ,
который указывается с помощью флага primary_key.
Первичный ключ может состоять из нескольких столбцов, в таком случае он будет являться составным. 
Также, обязательным атрибутом колонки является тип данных.
Типы данных описываются с помощью специальных объектов, представленных в пакете SQLAlchemy, таких, как Integer и String.
SQLAlchemy позволяет работать с десятками типов данных и создавать свои собственные, подробнее об этом можно узнать в документации:

    https://docs.sqlalchemy.org/en/14/core/type_basics.html
        
Атрибуты объекта Column позволяют предельно точно описывать каждый столбец.
Булевые параметры nullable -может ли ячейка быть NULLом, default - значение по умолчанию, autoincrement,
поведение при изменении/удаление элемента родительской таблицы, все это легко задается с помощью данных атрибутов.

В то время как класс Table хранит информацию о нашей БД, он ничего не говорит о логике объектов, что используются нашим приложением.
Для соответствия таблице users создадим класс User.
Метод __init__ — это конструктор класса. Будьте внимательны, SQLAlchemy не вызывает его напрямую.
Метод  __repr__ же вызывается при операторе print. 
Оба эти методы являются необязательными, определены, исключительно для понимания. 
В данном объекте можно определять любые функции, property и все это мы можем подружить с нашей таблицей в БД.
Сделать это можно с помощью встроенной в пакет функции mapper().
Функция mapper() создаст новый Mapper-объект и сохранит его для дальнейшего применения, ассоциирующегося с нашим классом. 
После того как мы описали наши модели и их отображения, необходимо вызвать метод метадаты create_all(). 
Этот метод проверит наличие таблиц в базе данных, и в случае их отсутствия, выполнит команду CREATE. 
Данный способ описания моделей является классический. 
Он позволяет разработчикам декомпозировать сущности на таблицу, пользовательский класс и mapper.
В таком случае мы соблюдаем принцип разделения задач. 
"""

In [ ]:
"""Однако чаще встречаются приложения, в которых данный принцип попросту ни к чему.
Для таких случаев используется альтернативный стиль представления моделей.
Этот стиль называется декларативный. С помощью него мы можем представить одновременно все три сущности при объявлении класса модели.
"""

from sqlalchemy import Column, Integer, String, create_engine
from sqlalchemy.orm import declarative_base, sessionmaker

engine = create_engine('sqlite:///sqlite_python.db')
Session = sessionmaker(bind=engine)
session = Session()
Base = declarative_base()


class User(Base):
   __tablename__ = 'user'
   id = Column(Integer, primary_key=True)
   name = Column(String(16), nullable=False)
   email = Column(String(60))
   login = Column(String(50), nullable=False)

   def __repr__(self):
       return f"{self.name}, {self.email}, {self.login}"

Base.metadata.create_all(engine)
 
"""
Вместо метадаты мы создаем базовый класс Base для декларативных определений классов.
Новому базовому классу будет присвоен класс метадаты, который создает соответствующие объекты Table 
и выполняет соответствующие mapper() вызовы на основе информации, 
декларативно предоставленной в ORM-модели. Теперь модели можно описывать просто наследуя их от объекта Base. 
Имя таблицы определяем с помощью атрибута __tablename__. 
После того как мы описали наши модели, необходимо вызвать метод метадаты create_all(), который находится внутри объекта Base.
При использовании декларативного стиля, мы также можем отдельно задавать конфигурация для объектов Table и Mapper.
Для этого нужно описать атрибуты __table_args__ и __mapper_args__ соответственно.
Эти атрибуты включают как позиционные, так и ключевые аргументы, 
которые обычно отправляются в конструкторы соответствующих объектов. 
Атрибуты могут быть указаны в одной из двух форм.
Один - как словарь, другой, как кортеж, где каждый аргумент позиционный.

__table_args__ = {'schema': 'some_schema'}
__table_args__ = (
       ForeignKeyConstraint(['id'], ['remote_table.id']),
       UniqueConstraint('foo'), )
 
В первом случае мы задали параметр конфигурации таблицы - схему, как словарь,
во втором - определили вторичный ключ и добавили проверку на уникальность поля, как кортеж позиционных аргументов.
Давайте добавим индекс на поле email для нашей таблицы User и добавим метод, например, на получение всех юзеров.
"""

from sqlalchemy import Column, Integer, String, Index
from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base, sessionmaker
from sqlalchemy.orm.exc import MultipleResultsFound, NoResultFound

Base = declarative_base()
engine = create_engine('sqlite:///sqlite_python.db')
Session = sessionmaker(bind=engine)
session = Session()


class User(Base):
   __tablename__ = 'user'
   __table_args__ = (Index('email_index', 'email'),)

   id = Column(Integer, primary_key=True)
   name = Column(String(16), nullable=False)
   email = Column(String(60))
   login = Column(String(50), nullable=False)

   def __repr__(self):
       return f"{self.name}, {self.email}, {self.login}"

   @classmethod
   def get_all_users(cls):
       return session.query(User).all()
Base.metadata.create_all(engine)
users = User.get_all_users()
 
"""
В методе get_all_users я описал SQL-запрос, который транслируется в базу данных как SELECT * FROM USER;
Давайте разберем его по подробнее. 
Session.query() возвращает Query объект, который является источником всех операторов SELECT, генерируемых ORM. 
Внутрь запроса мы передаем сущность, которая фактически будет идти после ключевого слова SELECT в запросе.
Сюда можно передать множество сущностей сразу, например, взять из одной модели только атрибут name,
передав в запрос User.name и через запятую описывать другие сущности. 
Теперь давайте разберемся с методами объекта Query. 
В моем примере используются метод all(), который возвращает результаты, представленные данным Query в виде списка.
Query.first() - вернет первую найденную строку, one() - одну строку и так далее.
Будьте внимательнее с методами запроса, некоторые из них в случае не целевого результата выбрасывают исключения.
Так, метод one() - выкинет ошибку NoResultFound, если не будет найдено ни одной строки и MultipleResultsFound в случае,
если строк по данному запросу больше одной.
Кстати, удобно использовать метод one_or_none, который вернет None, если запрос не вернет ни одной строки.
Список доступных методов по работе с запросами вы сможете найти в документации:
    
    https://docs.sqlalchemy.org/en/14/orm/query.html
        
Давайте добавим еще один метод по работе с нашей моделью.
Кстати, почему classmethod? 
Потому что мы хотим применять его не к конкретному экземпляру класса модели,
проще говоря не к конкретной строке в таблице, а в целом к таблице.
Получается набор методов модели будет представлять из себя некое хранилище наиболее частых запросов. 
Итак, вернемся к методу. Давайте добавим метод, на вход которого будет поступать email пользователя,
и по запросу мы будем получать сущность данного юзера.
"""

from sqlalchemy.orm.exc import MultipleResultsFound, NoResultFound
@classmethod
def get_user_by_email(email: str):
   try:
       user = session.query(User) \
           .filter(User.email == email).one()
       return user
   except NoResultFound:
       print(f"Пользователь c {email} отсутствует")
   except MultipleResultsFound:
       print("Ошибка уникальности индекса")
        
"""
Метод filter() добавляет в наш запрос оператор WHERE.
В нем через запятую можно описать весь набор условий, которые в итоге будут объединены оператором and.
"""